***
# ICAD Data Management
***

In [1]:
#Import libraries
import os, sys, json
import pandas as pd
import numpy as np
import toml

In [2]:
#Set Pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
wdir = os.getcwd()
wdir

'/Users/akshayranade/Documents/Personal/icad_data_management'

In [4]:
#Change directory to database operations to load all the methods
#Import all methods from database_operations
from database_operations.create_database_function import create_postgres_db
from database_operations.create_df_from_sql_function import create_df_from_sql
from database_operations.create_table_from_pandas_df_function import create_table_from_df

from data_cleaning.clean_column_names import clean_column_names
from data_cleaning.clean_strings import clean_string_data

from data_matching.get_master_school import get_standard_school

In [5]:
# Load config
config = toml.load(wdir + "/config.toml")
print (config)

{'postgres_credentials': {'username': 'postgres', 'password': '', 'host': '127.0.0.1', 'port': '5432'}, 'db_details': {'dbname': 'icad_db', 'school_master': 'school_master', 'all_students': 'all_students', 'compitative_exams': 'compitative_exams', 'cpa_results': 'cpa_results', 'exam_marks': 'exam_marks', 'admissions': 'admissions'}, 'file_locations': {'all_schools': '/Users/akshayranade/Documents/Personal/icad_data_management/data/sales/activity_report.csv'}}


In [6]:
#Create database in postgres called icad_db
create_postgres_db(config['postgres_credentials'], config['db_details']['dbname'])

Database already exists:  icad_db
PostgreSQL connection is closed



***
## Load all school data in a table
***

In [7]:
#Read all schools data from csv
all_schools_df = pd.read_csv(config["file_locations"]["all_schools"])

In [8]:
all_schools_df.head(10)

,institute_name,address1,address2,city,district,state,board,contact_no,email_address,category,branch_name
0,ABC Convent and High School,Manewada,Manewada,Nagpur,Nagpur,Maharashtra,State,9096300320,NaN,B,EAST CENTRE - NAGPUR
1,Adarsh Sanskar Vidyalaya Hasanbagh,Hasanbagh,Hasanbagh,Nagpur,Nagpur,Maharashtra,State,NaN,NaN,B,EAST CENTRE - NAGPUR
2,Adarsh Sanskar Vidyalaya CBSE Hudkeshwar,Hudkeshwar,Hudkeshwar,Nagpur,Nagpur,Maharashtra,State,7620264605,NaN,B,EAST CENTRE - NAGPUR
3,Adarsh Vidya Mandir,CA Road,CA Road,Nagpur,Nagpur,Maharashtra,State,NaN,NaN,B,EAST CENTRE - NAGPUR
4,Adarsh Vidyalaya,Umrer,Umrer,Umred,Nagpur,Maharashtra,State,NaN,NaN,B,EAST CENTRE - NAGPUR
5,Amit High School,Dighori,Dighori,Nagpur,Nagpur,Maharashtra,State,NaN,NaN,C,EAST CENTRE - NAGPUR
6,Ankush Classes,Near dada Saheb thakre High School ; Aashirwaad Nagar Hudkeshwar Sq,Aashirwaad Nagar,Nagpur,Nagpur,Maharashtra,State,9881015403,noid@emailid.com,B,EAST CENTRE - NAGPUR
7,Ashok Kanya Vidyalaya,Umrer,Umrer,Umred,Nagpur,Maharashtra,State,NaN,NaN,C,EAST CENTRE - NAGPUR
8,Ashok Vidyalaya Boys,Umrer,Umrer,Umred,Nagpur,Maharashtra,State,NaN,NaN,C,EAST CENTRE - NAGPUR
9,Baba Nanak Sidhi Hindi High School,Nandanwan,Nandanwan,Nagpur,Nagpur,Maharashtra,State,7122764288,babananaksindhihindihs@gmail.com,C,EAST CENTRE - NAGPUR


In [9]:
all_schools_df.dtypes

institute_name    object
 address1         object
 address2         object
 city             object
 district         object
 state            object
 board            object
 contact_no       object
 email_address    object
category          object
branch_name       object
dtype: object

In [10]:
all_schools_df.columns

Index(['institute_name', ' address1', ' address2', ' city', ' district',
       ' state', ' board', ' contact_no', ' email_address', 'category',
       'branch_name'],
      dtype='object')

In [11]:
#Clean column names
all_schools_df = clean_column_names(all_schools_df)

In [12]:
all_schools_df.columns

Index(['institute_name', 'address1', 'address2', 'city', 'district', 'state',
       'board', 'contact_no', 'email_address', 'category', 'branch_name'],
      dtype='object')

In [13]:
#Clean school names, addresses
all_schools_df['institute_name'] = all_schools_df['institute_name'].apply(lambda x: clean_string_data(x))
all_schools_df['address1'] = all_schools_df['address1'].apply(lambda x: clean_string_data(x))
all_schools_df['address2'] = all_schools_df['address2'].apply(lambda x: clean_string_data(x))
all_schools_df['address2'] = all_schools_df['address2'].apply(lambda x: clean_string_data(x))
all_schools_df['city'] = all_schools_df['city'].apply(lambda x: clean_string_data(x))
all_schools_df['state'] = all_schools_df['state'].apply(lambda x: clean_string_data(x))

In [14]:
all_schools_df.head()

,institute_name,address1,address2,city,district,state,board,contact_no,email_address,category,branch_name
0,ABC CONVENT AND HIGH SCHOOL,MANEWADA,MANEWADA,NAGPUR,Nagpur,MAHARASHTRA,State,9096300320,NaN,B,EAST CENTRE - NAGPUR
1,ADARSH SANSKAR VIDYALAYA HASANBAGH,HASANBAGH,HASANBAGH,NAGPUR,Nagpur,MAHARASHTRA,State,NaN,NaN,B,EAST CENTRE - NAGPUR
2,ADARSH SANSKAR VIDYALAYA CBSE HUDKESHWAR,HUDKESHWAR,HUDKESHWAR,NAGPUR,Nagpur,MAHARASHTRA,State,7620264605,NaN,B,EAST CENTRE - NAGPUR
3,ADARSH VIDYA MANDIR,CA ROAD,CA ROAD,NAGPUR,Nagpur,MAHARASHTRA,State,NaN,NaN,B,EAST CENTRE - NAGPUR
4,ADARSH VIDYALAYA,UMRER,UMRER,UMRED,Nagpur,MAHARASHTRA,State,NaN,NaN,B,EAST CENTRE - NAGPUR


In [15]:
# Define unique identifier for a school
all_schools_ids = all_schools_df.rename_axis('school_id').reset_index()
all_schools_ids['school_id'] = all_schools_ids['school_id'] + 1000

In [16]:
all_schools_ids.head()

,school_id,institute_name,address1,address2,city,district,state,board,contact_no,email_address,category,branch_name
0,1000,ABC CONVENT AND HIGH SCHOOL,MANEWADA,MANEWADA,NAGPUR,Nagpur,MAHARASHTRA,State,9096300320,NaN,B,EAST CENTRE - NAGPUR
1,1001,ADARSH SANSKAR VIDYALAYA HASANBAGH,HASANBAGH,HASANBAGH,NAGPUR,Nagpur,MAHARASHTRA,State,NaN,NaN,B,EAST CENTRE - NAGPUR
2,1002,ADARSH SANSKAR VIDYALAYA CBSE HUDKESHWAR,HUDKESHWAR,HUDKESHWAR,NAGPUR,Nagpur,MAHARASHTRA,State,7620264605,NaN,B,EAST CENTRE - NAGPUR
3,1003,ADARSH VIDYA MANDIR,CA ROAD,CA ROAD,NAGPUR,Nagpur,MAHARASHTRA,State,NaN,NaN,B,EAST CENTRE - NAGPUR
4,1004,ADARSH VIDYALAYA,UMRER,UMRER,UMRED,Nagpur,MAHARASHTRA,State,NaN,NaN,B,EAST CENTRE - NAGPUR


In [17]:
#Create a table for all schools using a generic function
create_table_from_df(config['postgres_credentials'], config['db_details']['dbname'], config['db_details']['school_master'], all_schools_ids)

Table successfully created: school_master


***
## Read all students data from various sheets, collate it to create a student master table

1. Student master table will have student name, school name and a unique student id
2. Even if same student appears in different sheets, their information will be captured only once in the student_master table
3. School names to be standardised based on the school_master table
***

## 1. LEAP data

In [18]:
# Load the Leap data
leap_data_df = pd.read_excel('/Users/akshayranade/Documents/Personal/icad_data_management/data/sales/1. 8, 9, 10, 11 Moving All Data.xlsx', '7,8,9,10- Leap 2023-24')

In [19]:
leap_data_df.head()

,Roll No,Student Name,City,Mobile 1,Mobile 2,School,Class
0,121574,Sidhant Bobade,Amravati Centre,NaN,NaN,"Dnyanmata High School Camp road, near IMA hall, Amravati",7th
1,117785,Parth Bonde,Amravati Centre,NaN,NaN,"K.K. Cambridge School, Amravati",7th
2,122267,Ninad Matkar,Amravati Centre,NaN,NaN,"Orchid City International School, Amravati",7th
3,121575,Rachit Ravi Tayade,Amravati Centre,NaN,NaN,"Dnyanmata High School Camp road, near IMA hall, Amravati",7th
4,117780,Pranjal P Kantute,Amravati Centre,NaN,NaN,"K.K. Cambridge School, Amravati",7th


In [20]:
#Clean column names
leap_data_df = clean_column_names(leap_data_df)

In [24]:
leap_data_df['school'] = leap_data_df['school'].apply(lambda x: str(x).replace(',', ' '))

In [25]:
#Clean student names, school names, class
leap_data_df['student_name'] = leap_data_df['student_name'].apply(lambda x: clean_string_data(x))
leap_data_df['city'] = leap_data_df['city'].apply(lambda x: clean_string_data(x))
leap_data_df['school'] = leap_data_df['school'].apply(lambda x: clean_string_data(x))
leap_data_df['class'] = leap_data_df['class'].apply(lambda x: str(x).replace('th', '')).astype(int)

In [26]:
leap_data_df.head()

,roll_no,student_name,city,mobile_1,mobile_2,school,class
0,121574,SIDHANT BOBADE,AMRAVATI CENTRE,NaN,NaN,DNYANMATA HIGH SCHOOL CAMP ROAD NEAR IMA HALL AMRAVATI,7
1,117785,PARTH BONDE,AMRAVATI CENTRE,NaN,NaN,KK CAMBRIDGE SCHOOL AMRAVATI,7
2,122267,NINAD MATKAR,AMRAVATI CENTRE,NaN,NaN,ORCHID CITY INTERNATIONAL SCHOOL AMRAVATI,7
3,121575,RACHIT RAVI TAYADE,AMRAVATI CENTRE,NaN,NaN,DNYANMATA HIGH SCHOOL CAMP ROAD NEAR IMA HALL AMRAVATI,7
4,117780,PRANJAL P KANTUTE,AMRAVATI CENTRE,NaN,NaN,KK CAMBRIDGE SCHOOL AMRAVATI,7


In [27]:
leap_data_df.value_counts('class')

class
10    20317
9      7330
8      4503
7      4034
Name: count, dtype: int64

In [28]:
leap_data_df.shape[0]

36184

In [29]:
#Read the school_master table
school_master = create_df_from_sql(connection_config=config['postgres_credentials'], dbname='icad_db', tbname='school_master')

In [30]:
#Get standard school names & ID's 
leap_data_df['standard_school_name'] = leap_data_df['school'].apply(lambda x: get_standard_school(x, school_master)[0])
leap_data_df['standard_school_addr'] = leap_data_df['school'].apply(lambda x: get_standard_school(x, school_master)[1])
leap_data_df['standard_school_city'] = leap_data_df['school'].apply(lambda x: get_standard_school(x, school_master)[2])
leap_data_df['standard_school_id'] = leap_data_df['school'].apply(lambda x: get_standard_school(x, school_master)[3])

In [31]:
leap_data_df['year'] = '2023-24'

In [32]:
leap_data_df.head()

,roll_no,student_name,city,mobile_1,mobile_2,school,class,standard_school_name,standard_school_addr,standard_school_city,standard_school_id,year
0,121574,SIDHANT BOBADE,AMRAVATI CENTRE,NaN,NaN,DNYANMATA HIGH SCHOOL CAMP ROAD NEAR IMA HALL AMRAVATI,7,DNYANMATA HIGH SCHOOL AMRAVATI,CAMP ROAD 444602,AMRAVATI,1111,2023-24
1,117785,PARTH BONDE,AMRAVATI CENTRE,NaN,NaN,KK CAMBRIDGE SCHOOL AMRAVATI,7,KK CAMBRIDGE SCHOOL,MARDI ROAD AMRAVATI,AMRAVATI,2170,2023-24
2,122267,NINAD MATKAR,AMRAVATI CENTRE,NaN,NaN,ORCHID CITY INTERNATIONAL SCHOOL AMRAVATI,7,ORCHID CITY INTERNATIONAL SCHOOL AMRAVATI,NAVSARI TO AMRAVATI WALGAON ROAD NAVSARI AMRAVATI,AMRAVATI,2239,2023-24
3,121575,RACHIT RAVI TAYADE,AMRAVATI CENTRE,NaN,NaN,DNYANMATA HIGH SCHOOL CAMP ROAD NEAR IMA HALL AMRAVATI,7,DNYANMATA HIGH SCHOOL AMRAVATI,CAMP ROAD 444602,AMRAVATI,1111,2023-24
4,117780,PRANJAL P KANTUTE,AMRAVATI CENTRE,NaN,NaN,KK CAMBRIDGE SCHOOL AMRAVATI,7,KK CAMBRIDGE SCHOOL,MARDI ROAD AMRAVATI,AMRAVATI,2170,2023-24


In [33]:
leap_data_df.shape[0]

36184

In [34]:
#Write the data in table so that it can be referenced in future
create_table_from_df(config['postgres_credentials'], config['db_details']['dbname'], 'stg_leap_data', leap_data_df)

Table successfully created: stg_leap_data


In [ ]:
#Read leap data from database
leap_data_df = create_df_from_sql(connection_config=config['postgres_credentials'], dbname='icad_db', tbname='stg_leap_data')

In [18]:
# leap_data_df.to_csv('/Users/akshayranade/Documents/Personal/icad_data_management/data/sales/cleaned_leap_data.csv')

In [13]:
school_str = 'DNYANMATA HIGH SCHOOL CAMP ROAD NEAR IMA HALL AMRAVATI'

In [20]:
school_master.head()

,school_id,institute_name,address1,address2,city,district,state,board,contact_no,email_address,category,branch_name
0,1000,ABC CONVENT AND HIGH SCHOOL,MANEWADA,MANEWADA,NAGPUR,Nagpur,MAHARASHTRA,State,9096300320,None,B,EAST CENTRE - NAGPUR
1,1001,ADARSH SANSKAR VIDYALAYA HASANBAGH,HASANBAGH,HASANBAGH,NAGPUR,Nagpur,MAHARASHTRA,State,None,None,B,EAST CENTRE - NAGPUR
2,1002,ADARSH SANSKAR VIDYALAYA CBSE HUDKESHWAR,HUDKESHWAR,HUDKESHWAR,NAGPUR,Nagpur,MAHARASHTRA,State,7620264605,None,B,EAST CENTRE - NAGPUR
3,1003,ADARSH VIDYA MANDIR,CA ROAD,CA ROAD,NAGPUR,Nagpur,MAHARASHTRA,State,None,None,B,EAST CENTRE - NAGPUR
4,1004,ADARSH VIDYALAYA,UMRER,UMRER,UMRED,Nagpur,MAHARASHTRA,State,None,None,B,EAST CENTRE - NAGPUR


In [21]:
school_master['school_name_add1'] = school_master['institute_name'] + ' ' + school_master['address1']
school_master['school_name_add2'] = school_master['institute_name'] + ' ' + school_master['address2']
school_master['school_name_add1_city'] = school_master['institute_name'] + ' ' + school_master['address1'] + ' ' + school_master['city']
school_master['school_name_add2_city'] = school_master['institute_name'] + ' ' + school_master['address2'] + ' ' + school_master['city']
school_master['school_name_full_add'] = school_master['institute_name'] + ' ' + school_master['address1'] + ' ' + school_master['address2'] + ' ' + school_master['city'] 

In [22]:
school_master.head()

,school_id,institute_name,address1,address2,city,district,state,board,contact_no,email_address,category,branch_name,school_name_add1,school_name_add2,school_name_add1_city,school_name_add2_city,school_name_full_add
0,1000,ABC CONVENT AND HIGH SCHOOL,MANEWADA,MANEWADA,NAGPUR,Nagpur,MAHARASHTRA,State,9096300320,None,B,EAST CENTRE - NAGPUR,ABC CONVENT AND HIGH SCHOOL MANEWADA,ABC CONVENT AND HIGH SCHOOL MANEWADA,ABC CONVENT AND HIGH SCHOOL MANEWADA NAGPUR,ABC CONVENT AND HIGH SCHOOL MANEWADA NAGPUR,ABC CONVENT AND HIGH SCHOOL MANEWADA MANEWADA NAGPUR
1,1001,ADARSH SANSKAR VIDYALAYA HASANBAGH,HASANBAGH,HASANBAGH,NAGPUR,Nagpur,MAHARASHTRA,State,None,None,B,EAST CENTRE - NAGPUR,ADARSH SANSKAR VIDYALAYA HASANBAGH HASANBAGH,ADARSH SANSKAR VIDYALAYA HASANBAGH HASANBAGH,ADARSH SANSKAR VIDYALAYA HASANBAGH HASANBAGH NAGPUR,ADARSH SANSKAR VIDYALAYA HASANBAGH HASANBAGH NAGPUR,ADARSH SANSKAR VIDYALAYA HASANBAGH HASANBAGH HASANBAGH NAGPUR
2,1002,ADARSH SANSKAR VIDYALAYA CBSE HUDKESHWAR,HUDKESHWAR,HUDKESHWAR,NAGPUR,Nagpur,MAHARASHTRA,State,7620264605,None,B,EAST CENTRE - NAGPUR,ADARSH SANSKAR VIDYALAYA CBSE HUDKESHWAR HUDKESHWAR,ADARSH SANSKAR VIDYALAYA CBSE HUDKESHWAR HUDKESHWAR,ADARSH SANSKAR VIDYALAYA CBSE HUDKESHWAR HUDKESHWAR NAGPUR,ADARSH SANSKAR VIDYALAYA CBSE HUDKESHWAR HUDKESHWAR NAGPUR,ADARSH SANSKAR VIDYALAYA CBSE HUDKESHWAR HUDKESHWAR HUDKESHWAR NAGPUR
3,1003,ADARSH VIDYA MANDIR,CA ROAD,CA ROAD,NAGPUR,Nagpur,MAHARASHTRA,State,None,None,B,EAST CENTRE - NAGPUR,ADARSH VIDYA MANDIR CA ROAD,ADARSH VIDYA MANDIR CA ROAD,ADARSH VIDYA MANDIR CA ROAD NAGPUR,ADARSH VIDYA MANDIR CA ROAD NAGPUR,ADARSH VIDYA MANDIR CA ROAD CA ROAD NAGPUR
4,1004,ADARSH VIDYALAYA,UMRER,UMRER,UMRED,Nagpur,MAHARASHTRA,State,None,None,B,EAST CENTRE - NAGPUR,ADARSH VIDYALAYA UMRER,ADARSH VIDYALAYA UMRER,ADARSH VIDYALAYA UMRER UMRED,ADARSH VIDYALAYA UMRER UMRED,ADARSH VIDYALAYA UMRER UMRER UMRED


In [23]:
school_master[school_master['institute_name'] == 'DNYANMATA HIGH SCHOOL AMRAVATI'].head()

,school_id,institute_name,address1,address2,city,district,state,board,contact_no,email_address,category,branch_name,school_name_add1,school_name_add2,school_name_add1_city,school_name_add2_city,school_name_full_add
111,1111,DNYANMATA HIGH SCHOOL AMRAVATI,CAMP ROAD 444602,AMRAVATI,AMRAVATI,AMRAVATI,MAHARASHTRA,State,7212663263,None,A+,AMRAVATI CENTRE,DNYANMATA HIGH SCHOOL AMRAVATI CAMP ROAD 444602,DNYANMATA HIGH SCHOOL AMRAVATI AMRAVATI,DNYANMATA HIGH SCHOOL AMRAVATI CAMP ROAD 444602 AMRAVATI,DNYANMATA HIGH SCHOOL AMRAVATI AMRAVATI AMRAVATI,DNYANMATA HIGH SCHOOL AMRAVATI CAMP ROAD 444602 AMRAVATI AMRAVATI


In [25]:
school_master[school_master['institute_name'] == 'KK CAMBRIDGE SCHOOL'].head()

,school_id,institute_name,address1,address2,city,district,state,board,contact_no,email_address,category,branch_name,school_name_add1,school_name_add2,school_name_add1_city,school_name_add2_city,school_name_full_add
1165,2162,KK CAMBRIDGE SCHOOL,MARDI ROAD AMRAVATI,AMRAVATI,AMRAVATI,AMRAVATI,MAHARASHTRA,CBSE,None,None,A+,AMRAVATI CENTRE,KK CAMBRIDGE SCHOOL MARDI ROAD AMRAVATI,KK CAMBRIDGE SCHOOL AMRAVATI,KK CAMBRIDGE SCHOOL MARDI ROAD AMRAVATI AMRAVATI,KK CAMBRIDGE SCHOOL AMRAVATI AMRAVATI,KK CAMBRIDGE SCHOOL MARDI ROAD AMRAVATI AMRAVATI AMRAVATI


In [26]:
school_str

'DNYANMATA HIGH SCHOOL CAMP ROAD NEAR IMA HALL AMRAVATI'

In [27]:
#Match a string with a school name
standard_school_name = ''

#Check for exact match with either of the columns
if school_str in school_master['school_name_full_add'].to_list():
    step = 1
    i = school_master['school_name_full_add'].index[school_master['school_name_full_add'].eq(school_str)][0]
    standard_school_name = school_master.loc[i, 'institute_name']
    standard_school_addr = school_master.loc[i, 'school_name_add1_city']
    standard_school_id = school_master.loc[i, 'school_id']
elif school_str in school_master['school_name_add2_city'].to_list():
    step = 2
    i = school_master['school_name_add2_city'].index[school_master['school_name_add2_city'].eq(school_str)][0]
    standard_school_name = school_master.loc[i, 'institute_name']
    standard_school_addr = school_master.loc[i, 'school_name_add1_city']
    standard_school_id = school_master.loc[i, 'school_id']
elif school_str in school_master['school_name_add1_city'].to_list():
    step = 3
    i = school_master['school_name_add1_city'].index[school_master['school_name_add1_city'].eq(school_str)][0]
    standard_school_name = school_master.loc[i, 'institute_name']
    standard_school_addr = school_master.loc[i, 'school_name_add1_city']
    standard_school_id = school_master.loc[i, 'school_id']
elif school_str in school_master['school_name_add2'].to_list():
    step = 4
    i = school_master['school_name_add2'].index[school_master['school_name_add2'].eq(school_str)][0]
    standard_school_name = school_master.loc[i, 'institute_name']
    standard_school_addr = school_master.loc[i, 'school_name_add1_city']
    standard_school_id = school_master.loc[i, 'school_id']
elif school_str in school_master['school_name_add1'].to_list():
    step = 5
    i = school_master['school_name_add1'].index[school_master['school_name_add1'].eq(school_str)][0]
    standard_school_name = school_master.loc[i, 'institute_name']
    standard_school_addr = school_master.loc[i, 'school_name_add1_city']
    standard_school_id = school_master.loc[i, 'school_id']
elif school_str in school_master['institute_name'].to_list():
    step = 6
    i = school_master['institute_name'].index[school_master['institute_name'].eq(school_str)][0]
    standard_school_name = school_master.loc[i, 'institute_name']
    standard_school_addr = school_master.loc[i, 'school_name_add1_city']
    standard_school_id = school_master.loc[i, 'school_id']
else:
    print('N')

N


In [28]:
standard_school_name

''

In [30]:
from difflib import SequenceMatcher
import jellyfish

from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
from nltk.metrics.distance  import edit_distance

import textdistance

from textdistance import JaroWinkler, Jaro, Cosine, LCSSeq 

In [31]:
master_schools = school_master['school_name_add1_city'].to_list()

match_list_jaro = []
match_list_cosine = []

for s in master_schools:
    match_list_jaro.append(textdistance.jaro_winkler.similarity(school_str, s))
    match_list_cosine.append(textdistance.cosine.similarity(school_str, s))

In [33]:
max(match_list_jaro)

0.9033730158730159

In [38]:
jaro_index = match_list_jaro.index(max(match_list_jaro))

In [40]:
school_master.loc[jaro_index, 'institute_name']

'DNYANMATA HIGH SCHOOL AMRAVATI'

In [34]:
max(match_list_cosine)

0.8728715609439694

In [41]:
cosine_index = match_list_cosine.index(max(match_list_cosine))

In [84]:
school_str = 'DNYANMATA HIGH SCHOOL CAMP ROAD NEAR IMA HALL AMRAVATI'
# standard_str = "DNYANMATA HIGH SCHOOL AMRAVATI CAMP ROAD 444602 AMRAVATI"


# school_str = 'SOMALWAR HIGH SCHOOL RAMDASPETH'
# standard_str = "SOMALWAR HIGH SCHOOL AND JR COLLEGE RAMDASPETH"

In [78]:
#Sequence matcher
s = SequenceMatcher(None, school_str, standard_str)

In [79]:
s.ratio()

0.8051948051948052

In [80]:
#Jellyfish levenstein (replacement based)
jellyfish.levenshtein_distance(school_str, standard_str)

15

In [81]:
#jellyfish jaro (replacement based)
jellyfish.jaro_similarity(school_str, standard_str)

0.8590462833099579

In [82]:
textdistance.jaro_winkler.similarity(school_str, standard_str)

0.9154277699859747

In [83]:
textdistance.cosine.similarity(school_str, standard_str)

0.820922069065183

In [76]:
1-jaccard_distance(set(ngrams(school_str, 3)), set(ngrams(standard_str, 3)))

0.38

In [ ]:
# A function to standardise the school name with reference to the schoool_master table
def standard_school_mapping(school_name_str) -> str:
    #Read school_master data
    school_master = create_df_from_sql(connection_config=config['postgres_credentials'], dbname='icad_db', tbname='school_master')
     

In [107]:
#Split the school name and address
leap_data_df['school'] = leap_data_df['school'].str.strip()

In [112]:
leap_data_df.head()

,roll_no,student_name,city,mobile_1,mobile_2,school,class
0,121574,Sidhant Bobade,Amravati Centre,NaN,NaN,"Dnyanmata High School Camp road, near IMA hall...",7th
1,117785,Parth Bonde,Amravati Centre,NaN,NaN,"K.K. Cambridge School, Amravati",7th
2,122267,Ninad Matkar,Amravati Centre,NaN,NaN,"Orchid City International School, Amravati",7th
3,121575,Rachit Ravi Tayade,Amravati Centre,NaN,NaN,"Dnyanmata High School Camp road, near IMA hall...",7th
4,117780,Pranjal P Kantute,Amravati Centre,NaN,NaN,"K.K. Cambridge School, Amravati",7th


In [122]:
leap_data_df['school_name'] = leap_data_df['school'].apply(lambda x: str(x).split(',')[0] if len(str(x).split(',')) >= 1 else None)

In [124]:
leap_data_df['school_address'] = leap_data_df['school'].apply(lambda x: [' ' + str(x).split(',')[i] for i in (1, len(str(x).split(',')))] if len(str(x).split(',')) > 1 else None)

IndexError: list index out of range

In [115]:
leap_data_df.head()

,roll_no,student_name,city,mobile_1,mobile_2,school,class,school_name
0,121574,Sidhant Bobade,Amravati Centre,NaN,NaN,"Dnyanmata High School Camp road, near IMA hall...",7th,Dnyanmata High School Camp road
1,117785,Parth Bonde,Amravati Centre,NaN,NaN,"K.K. Cambridge School, Amravati",7th,K.K. Cambridge School
2,122267,Ninad Matkar,Amravati Centre,NaN,NaN,"Orchid City International School, Amravati",7th,Orchid City International School
3,121575,Rachit Ravi Tayade,Amravati Centre,NaN,NaN,"Dnyanmata High School Camp road, near IMA hall...",7th,Dnyanmata High School Camp road
4,117780,Pranjal P Kantute,Amravati Centre,NaN,NaN,"K.K. Cambridge School, Amravati",7th,K.K. Cambridge School


In [102]:
leap_data_df[leap_data_df['school_name'] == 'None'].head(40)

,roll_no,student_name,city,mobile_1,mobile_2,school,class,school_name,school_split
